<a href="https://colab.research.google.com/github/patrikviszt/unsupervisedLearning/blob/main/movieRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.impute import SimpleImputer
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import warnings

In [6]:
from google.colab import files
uploaded = files.upload()

Saving allbut.pl to allbut.pl
Saving mku.sh to mku.sh
Saving README to README
Saving u.data to u.data
Saving u.genre to u.genre
Saving u.info to u.info
Saving u.item to u.item
Saving u.occupation to u.occupation
Saving u.user to u.user
Saving u1.base to u1.base
Saving u1.test to u1.test
Saving u2.base to u2.base
Saving u2.test to u2.test
Saving u3.base to u3.base
Saving u3.test to u3.test
Saving u4.base to u4.base
Saving u4.test to u4.test
Saving u5.base to u5.base
Saving u5.test to u5.test
Saving ua.base to ua.base
Saving ua.test to ua.test
Saving ub.base to ub.base
Saving ub.test to ub.test


In [8]:
data_column = ['user_id', 'item_id', 'rating', 'timestamp']
userdata = pd.read_csv('u.data', sep='\t', names=data_column)
userdata.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [10]:
user_column = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
user = pd.read_csv('u.user', sep ='|', names=user_column)
user.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [14]:
item_column = ['item_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL']
item = pd.read_csv('u.item', sep='|',names=item_column, encoding='latin1', usecols=[0,1,2,3,4])
item['release_date'] = pd.to_datetime(item['release_date'], format='%d-%b-%Y', errors='coerce')
item['video_release_date'] = pd.to_datetime(item['video_release_date'], format='%d-%b-%Y', errors='coerce')

item.head()

,item_id,movie_title,release_date,video_release_date,IMDb_URL
0,1,Toy Story (1995),1995-01-01,NaT,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),1995-01-01,NaT,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),1995-01-01,NaT,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),1995-01-01,NaT,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),1995-01-01,NaT,http://us.imdb.com/M/title-exact?Copycat%20(1995)
